# Finite Difference Methods

Explicit, Implicit and Crank-Nicolson are the three popular approaches of FDM. The explicit
methods are simple to implement, but it does not always converge and largely depends on the size
of the time and asset step. Explicit methods are unstable when compared to other two methods.
Finite Difference approach is peferred for low dimensional problem, usually upto 4 dimensions.

# Differentiation Using The Grid

* $ S = i \delta s $
* $ t = T - k \delta t $
* $ V^i_k = (i \delta S, T - k \delta t) $

# Approximating Greeks

Using the greeks terms the Black–Scholes equation can be rewritten as:
$$ \Theta + \frac{1}{2} \sigma^2 S^2 \Gamma + r S \Delta - r V = 0 $$
    
$$ \Theta = \frac{\partial V}{\partial t} \approx \frac { V_i^k - V_i^{k+1} } { \delta t } $$
$$ \Delta = \frac{\partial V}{\partial S} \approx \frac { V_{i+1}^k - V_i^{k-1} } { 2 \delta S } $$
$$ \Gamma = \frac{\partial^2 V}{\partial S^2} \approx \frac { V_{i+1}^k - 2V_i^k + V_i^{k-1} } { \delta S^2 } $$

In [41]:
# Importing libraries
import pandas as pd
import numpy as np
# plotting
import cufflinks as cf
cf.set_config_file(offline=True)
# rendering
# import plotly.io as pio

# pio.renderers.default = 'notebook_connected'
# Set max row and columns to 50
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Option Pricing Techniques

In [42]:
# Specify the parameters for FDM

T = 1 # time to maturity in years
E = 100 # strike price
r = .05 # riskfree rate
vol = .20 # volatility
CALL, PUT = 1, -1
Flag = CALL # Flag = 1 for call, -1 for puts
NAS = 20 # number of asset steps
ds = 2* E / NAS # asset step size
dt = (0.9/vol**2/NAS**2) # time step size, for stability, 0.9 for strict lower
NTS = int(T / dt) + 1 # number of time steps
dt = T / NTS # time step size [Expiration as int # of time steps away]

In [43]:
## Generate Grid

In [44]:
# Create asset steps i*ds
s = np.arange(0, (NAS+1)*ds,ds)
s

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200.])

In [45]:
# Create time steps k*dt
t = T-np.arange(NTS*dt,-dt,-dt)
t

array([0.        , 0.05555556, 0.11111111, 0.16666667, 0.22222222,
       0.27777778, 0.33333333, 0.38888889, 0.44444444, 0.5       ,
       0.55555556, 0.61111111, 0.66666667, 0.72222222, 0.77777778,
       0.83333333, 0.88888889, 0.94444444, 1.        ])

In [46]:
# Verify the steps size
s.shape, t.shape

((21,), (19,))

In [47]:
# Initialize the grid with zeros
grid = np.zeros((len(s),len(t)))
# Subsume the grid points into a dataframe
# with asset price as index and time steps as columns
grid = pd.DataFrame(grid, index=s, columns=np.around(t,3))
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 4.2 Specify boundary Condition

For $ t = T $ (payoff)
* $ V_i^0 = max(i \delta s - E, 0) $

In [48]:
# Set Final or Initial condition at Expiration
if Flag == CALL:
    grid.iloc[:,0] = np.maximum(s - E, 0)
else:
    grid.iloc[:,0] = np.maximum(E - s, 0)

In [49]:
# k is counter
for k in range(1, len(t)):
    # Set boundary condition at S = 0
    grid.iloc[0,k] = grid.iloc[0,k-1] * (1-r*dt) # ds = rsdt + sigma*sdx, s= 0, ds = 0

    # Set boundary condition at S = infinity # gamma = 0, so you can linearly extract

    grid.iloc[len(s)-1,k] = 2*grid.iloc[len(s)-2,k] - grid.iloc[len(s)-3,k]

In [50]:
# Verify the grid
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Fill the Grid

General (for both Put and Call) conditions

For $ S = 0 $
* $ V_0^k = (1 - r \delta t)V_0^{k-1} $

For $ S = \infty $
* $ V_{max}^k = 2V_{max-1}^k - V_{max-2}^k $

In [51]:
# k is counter
for k in range(1, len(t)):
    for i in range(1,len(s)-1):
        delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
        gamma = (grid.iloc[i+1,k-1]-2*grid.iloc[i,k-1]+grid.iloc[i-1,k-1]) / (ds**2)
        theta = (-0.5* vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i,k-1])
        
        grid.iloc[i,k] = grid.iloc[i,k-1] - (theta*dt)
    
    # Set boundary condition at S = 0
    grid.iloc[0,k] = grid.iloc[0,k-1] * (1-r*dt) # ds = rsdt + sigma*sdx, s= 0, ds = 0

    # Set boundary condition at S = infinity # gamma = 0, so you can linearly extract

    grid.iloc[len(s)-1,k] = 2*grid.iloc[len(s)-2,k] - grid.iloc[len(s)-3,k]

# Round grid values to 2 decimals
grid = np.around(grid,3)

In [52]:
# Output the option values
# grid.iloc[0:15,:]
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
20.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
30.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
40.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.001,0.002,0.003,0.004,0.006
60.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.003,0.004,0.007,0.011,0.016,0.023,0.031,0.041,0.053,0.066
70.0,0.0,0.000,0.000,0.000,0.001,0.003,0.008,0.016,0.028,0.045,0.067,0.094,0.126,0.164,0.207,0.255,0.308,0.367,0.430
80.0,0.0,0.000,0.000,0.011,0.037,0.080,0.141,0.218,0.310,0.416,0.534,0.662,0.799,0.944,1.096,1.253,1.416,1.583,1.754
90.0,0.0,0.000,0.128,0.336,0.592,0.878,1.182,1.495,1.812,2.131,2.450,2.766,3.080,3.392,3.700,4.004,4.306,4.604,4.899


# Visualize the payoff

In [54]:
# Plot Call Option Payoff
grid.iloc[0:15,:].iplot(kind = 'surface', title='Call Option values by Explicit FDM')

# Bilinear Interpolation

$$ \frac{\sum^4_{i=1} A_i V_i}{\sum^4_{i=1} A_i } $$

In [57]:
def bilinear_interpolation(asset_price, ttm, df: pd.DataFrame):
    # Find relevant rows and columns
    col1 = df.columns[df.columns < ttm][-1]
    col2 = df.columns[df.columns >= ttm][0]
    row1 = df.index[df.index < asset_price,][-1]
    row2 = df.index[df.index >= asset_price,][0]

    # Define points and areas
    V = [df.loc[row1, col1], df.loc[row1, col2], df.loc[row2, col2], df.loc[row2, col1]]
    A = [
        (row2 - asset_price) * (col2 - ttm),
        (row2 - asset_price) * (ttm - col1),
        (asset_price - row1) * (ttm - col1),
        (asset_price - row1) * (col2 - ttm),
    ]

    # Interpolate values
    return sum(np.array(V) * np.array(A)) / sum(np.array(A))


# Option value, approximated
bilinear_interpolation(105, 0.25, grid)

7.98375

# User Defined Function

In [64]:
def efdm_grid(Strike, Volatility, Rate, TTM, NAS, Flag=CALL):
    # Specify Flag as 1 for calls and -1 for puts

    ds = 2 * Strike / NAS  # asset step size
    dt = 0.9 / Volatility**2 / NAS**2  # for stability

    NTS = int(TTM / dt) + 1  # time step size, alternatively use fixed size 10 on stability issue
    dt = TTM / NTS  # time step
    s = np.arange(0, (NAS + 1) * ds, ds)
    t = TTM - np.arange(NTS * dt, -dt, -dt)
    
    # Initialize the grid with zeros
    grid = np.zeros((len(s), len(t)))
    grid = pd.DataFrame(grid, index=s, columns=np.around(t, 2))

    # Set boundary condition at Expiration
    grid.iloc[:, 0] = np.abs(np.maximum(Flag * (s - Strike), 0))
    for k in range(1, len(t)):
        for i in range(1, len(s) - 1):
            delta = (grid.iloc[i + 1, k - 1] - grid.iloc[i - 1, k - 1]) / (2 * ds)
            gamma = (
                grid.iloc[i + 1, k - 1] - 2 * grid.iloc[i, k - 1] + grid.iloc[i - 1, k - 1]
            ) / (ds**2)
            theta = (
                (-0.5 * vol**2 * s[i] ** 2 * gamma)
                - (r * s[i] * delta)
                + (r * grid.iloc[i, k - 1])
            )
            grid.iloc[i, k] = grid.iloc[i, k - 1] - dt * theta

        # Set boundary condition at S = 0
        grid.iloc[0, k] = grid.iloc[0, k - 1] * (1 - r * dt)

        # Set boundary condition at S = infinity
        grid.iloc[len(s) - 1, k] = abs(2 * (grid.iloc[len(s) - 2, k]) - grid.iloc[len(s) - 3, k])

    # round grid values to 4 decimals
    return np.around(grid,2)

In [65]:
# Call the function for put option
fdm_puts = efdm_grid(100, 0.2, 0.05, 1, 20, Flag=PUT)
fdm_puts

,0.00,0.06,0.11,0.17,0.22,0.28,0.33,0.39,0.44,0.50,0.56,0.61,0.67,0.72,0.78,0.83,0.89,0.94,1.00
0.0,100.0,99.72,99.45,99.17,98.89,98.62,98.34,98.07,97.80,97.53,97.26,96.99,96.72,96.45,96.18,95.91,95.65,95.38,95.12
10.0,90.0,89.72,89.45,89.17,88.89,88.62,88.34,88.07,87.80,87.53,87.26,86.99,86.72,86.45,86.18,85.91,85.65,85.38,85.12
20.0,80.0,79.72,79.45,79.17,78.89,78.62,78.34,78.07,77.80,77.53,77.26,76.99,76.72,76.45,76.18,75.91,75.65,75.38,75.12
30.0,70.0,69.72,69.45,69.17,68.89,68.62,68.34,68.07,67.80,67.53,67.26,66.99,66.72,66.45,66.18,65.91,65.65,65.38,65.12
40.0,60.0,59.72,59.45,59.17,58.89,58.62,58.34,58.07,57.80,57.53,57.26,56.99,56.72,56.45,56.18,55.91,55.65,55.38,55.12
50.0,50.0,49.72,49.45,49.17,48.89,48.62,48.34,48.07,47.80,47.53,47.26,46.99,46.72,46.45,46.18,45.92,45.65,45.39,45.12
60.0,40.0,39.72,39.45,39.17,38.89,38.62,38.35,38.07,37.80,37.53,37.26,36.99,36.73,36.46,36.20,35.94,35.69,35.43,35.18
70.0,30.0,29.72,29.45,29.17,28.89,28.62,28.35,28.09,27.83,27.57,27.32,27.08,26.84,26.61,26.39,26.17,25.96,25.75,25.55
80.0,20.0,19.72,19.45,19.18,18.93,18.70,18.49,18.29,18.11,17.94,17.79,17.65,17.52,17.39,17.28,17.17,17.06,16.96,16.87
90.0,10.0,9.72,9.57,9.50,9.49,9.50,9.53,9.57,9.61,9.66,9.71,9.75,9.80,9.84,9.88,9.92,9.95,9.98,10.01


In [66]:
# Visualize the plot for put option
fdm_puts.iplot(kind='surface', title='Put Option values by Explicit FDM')

# Convergence Analysis

In [90]:
# Iterate over asset steps (NAS)
nas_list = np.arange(10, 60+1, 10)
fdmoption = []
for i in nas_list:
    fdmoption.append(efdm_grid(100,0.2,0.05,1,i).loc[100,1])
fdmoption

[9.51, 10.26, 10.37, 10.4, 10.42, 10.43]

In [91]:
# Call black scholes class
from src.blackscholes import BS

# Instantiate black scholes object
option = BS(100,100,0.05,1,0.20)
bsoption = round(option.callPrice,2)
bsoption = bsoption.repeat(len(nas_list))

# Range of option price
bsoption

array([10.45, 10.45, 10.45, 10.45, 10.45, 10.45])

In [92]:
# Subsume into dataframe
df = pd.DataFrame(list(zip(bsoption,fdmoption)), columns=['BS', 'FDM'], index=nas_list)

df['dev'] = df['FDM'] - df['BS']
df['% dev'] = round(df['dev'] / df['BS'] * 100.,2)

# Output
print("BS - FDM Convergence over NAS")
df

BS - FDM Convergence over NAS


,BS,FDM,dev,% dev
10,10.45,9.51,-0.94,-9.00
20,10.45,10.26,-0.19,-1.82
30,10.45,10.37,-0.08,-0.77
40,10.45,10.40,-0.05,-0.48
50,10.45,10.42,-0.03,-0.29
60,10.45,10.43,-0.02,-0.19
